## Lecture 11:


* We are going to create toy inception modules and then use them in our model.
* Everything else remains same as the lecture 10.
* The dataset to be used is the **_MINST Dataset_** given by pytorch package


_It is worth noting that we can create a new network component as a class and then use that object in our core network. This is really a very neat and useful implementation_

In [1]:
# Importing the stock libraries

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

In [2]:
# Creating the CUDA environment

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
# Creating the dataset for the model

train_dataset = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor(), download=True)

In [4]:
# Creating the dataloader for the input to the model

train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [5]:
# This is where we are defining the inception component, which is a network in itself. But once defined as a class we can then use the object ob this class in other networks as a reusable element. 

class InceptionM(torch.nn.Module):

    # Initialize
    def __init__(self, in_channels):
        super(InceptionM, self).__init__()
        self.branch_01_02 = torch.nn.Conv2d(in_channels, 24, kernel_size=1)
        self.branch_02_01 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        
        self.branch_03_01 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch_03_02 = torch.nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch_04_01 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch_04_02 = torch.nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch_04_03 = torch.nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
        
    def forward(self, x):
        branch_01_02 = self.branch_01_02(torch.nn.functional.avg_pool2d(x, kernel_size=3, stride=1, padding=1))
        
        branch_02_01 = self.branch_02_01(x)
        
        branch_03_01 = self.branch_03_01(x)
        branch_03_02 = self.branch_03_02(branch_03_01)

        branch_04_01 = self.branch_04_01(x)
        branch_04_02 = self.branch_04_02(branch_04_01)
        branch_04_03 = self.branch_04_03(branch_04_02)

        outputs = [branch_01_02,branch_02_01,branch_03_02,branch_04_03]

        return torch.cat(outputs, 1)
        

In [6]:
# Creating the Model

class Model(torch.nn.Module):

    ## Initiatize
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88,20,kernel_size=5)

        self.incep_1 = InceptionM(in_channels=10)
        self.incep_2 = InceptionM(in_channels=20)

        self.l1 = torch.nn.Linear(1408, 10)

        self.max = torch.nn.MaxPool2d(2)


    ## Forward
    def forward(self, x):
        in_size = x.size(0)
        x = torch.nn.functional.relu(self.max(self.conv1(x)))
        x = self.incep_1(x)
        x = torch.nn.functional.relu(self.max(self.conv2(x)))
        x = self.incep_2(x)
        x = x.view(in_size, -1)
        x = self.l1(x)
        return torch.nn.functional.log_softmax(x)

model = Model()
model.to(device)

Model(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(88, 20, kernel_size=(5, 5), stride=(1, 1))
  (incep_1): InceptionM(
    (branch_01_02): Conv2d(10, 24, kernel_size=(1, 1), stride=(1, 1))
    (branch_02_01): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch_03_01): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch_03_02): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch_04_01): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch_04_02): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch_04_03): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (incep_2): InceptionM(
    (branch_01_02): Conv2d(20, 24, kernel_size=(1, 1), stride=(1, 1))
    (branch_02_01): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch_03_01): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch_03_02): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), 

In [7]:
# Define the loss function and optimizer

criterion = torch.nn.CrossEntropyLoss(reduction='mean')
optimus = torch.optim.SGD(model.parameters(), lr = 0.01, momentum=0.5)

In [8]:
# This is the code for training

def train(epoch):
    model.train()
    for _, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        predict = model(inputs)

        # loss
        loss = criterion(predict, labels)
        if _%500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        # zero optimus,back propagation, update optimus
        optimus.zero_grad()
        loss.backward()
        optimus.step()


In [9]:
# This code is for the validation step

def valid(test_dataloader):
    model.eval()
    total=0; n_correct =0 
    with torch.no_grad():
        for _, data in enumerate(test_dataloader,0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            if _%500 == 0:
                print(f'Validation completed for {_} samples')

            predict = model(inputs)
            total += labels.size(0)
            big_val, big_idx = torch.max(predict.data, dim=1)
            n_correct += (big_idx==labels).sum().item()

    return (n_correct*100)/total

In [10]:
# Calling the training function

print('Starting the training')

for epoch in range(0,2):
    train(epoch)

print('Training Completed')

print('+++++++++++++++++=================+++++++++++++++++=================++++++++++++++++')

print('Starting the testing')
accu = valid(test_dataloader)
print(f'The accuracy of the model is: {accu}')

Starting the training
Epoch: 0, Loss:  2.3154919147491455
Epoch: 0, Loss:  0.5073910355567932
Epoch: 0, Loss:  0.13816455006599426
Epoch: 0, Loss:  0.17782428860664368
Epoch: 1, Loss:  0.0799398273229599
Epoch: 1, Loss:  0.021845880895853043
Epoch: 1, Loss:  0.24345053732395172
Epoch: 1, Loss:  0.059964992105960846
Training Completed
+++++++++++++++++=================+++++++++++++++++=================++++++++++++++++
Starting the testing
Validation completed for 0 samples
The accuracy of the model is: 97.76
